In [1481]:
import pandas as pd
import numpy as np
import os

In [1482]:
#pip install pandas

In [1483]:
# Helper Methods

def generate_column_info(df):
    cls = []
    nullCount = []
    nonNullCount = []
    nullsPct = []
    uniqCount = []
    dataType = []
    for i,col in enumerate(df.columns):
        cls.append(col)
        nullCount.append(df[col].isnull().sum())
        nonNullCount.append(len(df)-df[col].isnull().sum())
        nullsPct.append((df[col].isnull().sum())*(100)/len(df))
        uniqCount.append(df[col].nunique())
        dataType.append(df[col].dtype)
        
    column_info = pd.DataFrame(
        {'ColumnName': cls,
         'NullCount': nullCount,
         'NonNullCount': nonNullCount,
         'NullPercent': nullsPct,
         'UniqueValueCount': uniqCount,
         'DataType':dataType
        })
    return(column_info)

## Data

In [1484]:
abbr_dict = {
    'flight_Liquefied Natural Gas ImpExp Equipment.xls':'LNG_IMEX',
    'flight_Liquefied Natural Gas Storage.xls':'LNG_STOR',
    'flight_Natural Gas Local Distribution Companies.xls':'NG_LDC',
    'flight_Natural Gas Processing.xls':'NG_PROC',
    'flight_Natural Gas TransmissionCompression.xls':'NG_COMP',
    'flight_Offshore Petroleum & Natural Gas Production.xls':'PROD_OFF',
    'flight_Onshore Natural Gas Transmission Pipelines.xls':'NG_PIPE',
    'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls':'NG_GNB',
    'flight_Onshore Petroleum & Natural Gas Production.xls':'PROD_ON',
    'flight_Other Petroleum & Natural Gas Systems.xls':'ONG_OTH',
    'flight_Underground Natural Gas Storage.xls':'NG_STOR',
}

sub_sector_dict = {
    'NG_LDC':'NaturalGasDistribution',
    'PROD_OFF':'OffshoreProduction',
    'PROD_ON':'OnshoreProduction',
}

In [1485]:
#pip install pyxlsb

In [1486]:
#directory_path = 'C://Users//shashank.bekal//PythonWorkArea//Jnana-Research//OKC-Facilities//files'
directory_path = './/files'
all_files = os.listdir(directory_path)
xls_files = [file for file in all_files if file.endswith('.xls')]

print(xls_files)

['flight_Liquefied Natural Gas ImpExp Equipment.xls', 'flight_Liquefied Natural Gas Storage.xls', 'flight_Natural Gas Local Distribution Companies.xls', 'flight_Natural Gas Processing.xls', 'flight_Natural Gas TransmissionCompression.xls', 'flight_Offshore Petroleum & Natural Gas Production.xls', 'flight_Onshore Natural Gas Transmission Pipelines.xls', 'flight_Onshore Petroleum & Natural Gas Gathering & Boosting.xls', 'flight_Onshore Petroleum & Natural Gas Production.xls', 'flight_Other Petroleum & Natural Gas Systems.xls', 'flight_Underground Natural Gas Storage.xls']


In [1487]:
facilities_master_df = pd.DataFrame()
for file_name in xls_files:
    file_dict = pd.read_excel(f'.//files/{file_name}',sheet_name=None,skiprows=6)
    file_df = pd.concat(file_dict, ignore_index=True)
    file_df["Abbr"] = abbr_dict[file_name]
    #print((file_name))
    facilities_master_df = pd.concat([facilities_master_df,file_df])
#print('---')
print(len(facilities_master_df))

C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a 

30781


C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)
C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\2884616571.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  file_df = pd.concat(file_dict, ignore_index=True)


In [1488]:
#df

In [1489]:
required_subsectors = ['PROD_OFF','PROD_ON']

In [1490]:
# filter by 3 required subsectors
facilities_master_df = facilities_master_df[facilities_master_df['Abbr'].isin(required_subsectors)]
facilities_master_df['sub_sector'] = facilities_master_df['Abbr'].map(sub_sector_dict)
facilities_master_df = facilities_master_df.rename(columns={'GHG QUANTITY (METRIC TONS CO2e)':'FACILITY_WIDE_GHG_QUANTITY'})
facilities_master_df = facilities_master_df.rename(columns={'GHGRP ID':'facility_id','REPORTING YEAR':'reporting_year'})
facilities_master_df = facilities_master_df.drop(['FACILITY NAME','SUBPARTS'],axis=1)

In [1491]:
#generate_column_info(facilities_master_df)

In [1492]:
#facilities_master_df

## Facilities Metadata

In [1493]:
# facilities_metadata_df = pd.read_csv('rlps_ghg_emitter_facilities.csv')
# facilities_metadata_df = facilities_metadata_df[['facility_id','year','epa_verified']]
# facilities_metadata_df

In [1494]:
# facilities_metadata_df.columns

In [1495]:
# facilities_metadata_df[facilities_metadata_df.duplicated(subset=['facility_id','year'])]

In [1496]:
# facilities_master_df = pd.merge(facilities_master_df,facilities_metadata_df,how='left',left_on=['GHGRP ID','REPORTING YEAR'],right_on=['facility_id','year'])

In [1497]:
# facilities_master_df

In [1498]:
# facilities_master_df.columns

In [1499]:
# generate_column_info(facilities_master_df)

## Combustion Sources

In [1500]:
#emissions_by_unit_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx',engine="openpyxl", sheet_name="UNIT_DATA")
c_df = pd.read_excel('emissions_by_unit_and_fuel_type_c_d_aa.xlsx', sheet_name="UNIT_DATA",skiprows=6)
c_df['Reporting Year'] = c_df['Reporting Year'].astype(int)
c_df['Facility Id'] = c_df['Facility Id'].astype(int)
c_df = c_df.rename(columns={'Facility Id':'facility_id','Facility Name':'facility_name','Reporting Year':'reporting_year'})
c_df = c_df.rename(columns={'Unit Type':'reporting_category','Unit CO2 emissions (non-biogenic) ':'total_co2_emissions','Unit Methane (CH4) emissions ':'total_ch4_emissions'})
c_df = c_df.rename(columns={'Unit Nitrous Oxide (N2O) emissions ':'total_n2o_emissions'})
c_df = c_df.drop(columns=['Unit Reporting Method','City','Primary NAICS Code','FRS Id','Industry Type (subparts)','Industry Type (sectors)','Unit Biogenic CO2 emissions (metric tons)'])
c_df = c_df.rename(columns={'Unit Maximum Rated Heat Input Capacity (mmBTU/hr)':'maximum_rated_heat_input_capacity','State':'state'})
#c_df = c_df[c_df['reporting_category'].notna()]
c_df['subpart'] = 'SubpartC'
c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = c_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
c_df

,facility_id,facility_name,state,reporting_year,Unit Name,reporting_category,maximum_rated_heat_input_capacity,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,subpart
0,1001155,1500 South Tibbs LLC d/b/a Aurorium Indianapol...,IN,2023,GP-1,NaN,91.8,64740.5,30.50,36.356,SubpartC
1,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.00,OCS (Other combustion source),30.0,3304.7,1.50,1.788,SubpartC
2,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.01,OCS (Other combustion source),30.0,0.2,0.00,0.000,SubpartC
3,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2018,CP-03.02,OCS (Other combustion source),30.0,175.0,0.00,0.000,SubpartC
4,1012147,17Z Gas Plant - Chevron USA Inc.,CA,2017,CP-03.00,OCS (Other combustion source),30.0,9106.1,4.25,5.066,SubpartC
...,...,...,...,...,...,...,...,...,...,...,...
268773,1003318,ZYBACH CRYOGENIC PLANT,TX,2015,GP-Zybach,OCS (Other combustion source),32.0,33757.8,16.00,19.072,SubpartC
268774,1003318,ZYBACH CRYOGENIC PLANT,TX,2014,GP-Zybach,OCS (Other combustion source),32.0,32901.2,15.50,18.476,SubpartC
268775,1003318,ZYBACH CRYOGENIC PLANT,TX,2013,GP-Zybach,OCS (Other combustion source),32.0,42700.7,20.25,24.138,SubpartC
268776,1003318,ZYBACH CRYOGENIC PLANT,TX,2012,GP-Zybach,OCS (Other combustion source),32.0,46840.7,22.00,26.224,SubpartC


# Emission Source

In [1501]:
w_df = pd.read_csv('ef_w_emissions_source_ghg.csv',encoding = 'ISO-8859-1')
print(len(w_df))
w_df['reporting_year'] = w_df['reporting_year'].astype(int)
w_df['facility_id'] = w_df['facility_id'].astype(int)
w_df = w_df.drop(columns=['bamm_desc_source_summary','bamm_indicator_source_summary','industry_segment'], axis=1)
w_df = w_df.rename(columns={'total_reported_co2_emissions':'total_co2_emissions'})
w_df = w_df.rename(columns={'total_reported_ch4_emissions':'total_ch4_emissions'})
w_df = w_df.rename(columns={'total_reported_n2o_emissions':'total_n2o_emissions'})
w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = w_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
w_df['total_ch4_emissions'] = w_df['total_ch4_emissions'] * 25
w_df['total_n2o_emissions'] = w_df['total_n2o_emissions'] * 298
w_df['subpart'] = 'SubpartW'
w_df

503060


,basin_associated_with_facility,facility_id,facility_name,reporting_category,reporting_year,total_ch4_emissions,total_co2_emissions,total_n2o_emissions,subpart
0,NaN,1003447,UGC - DRAGON TRAIL GAS PLANT - 08-103-00036,Enhanced Oil Recovery Hydrocarbon Liquids [98....,2016,0.00,0.0,0.0,SubpartW
1,NaN,1004777,Great Lakes Gas Transmission Pipeline Shevlin ...,Completions and Workovers with Hydraulic Fract...,2023,0.00,0.0,0.0,SubpartW
2,"220 - Gulf Coast Basin (LA, TX)",1009706,"SilverBow Resources, LLC - Basin 220",Completions and Workovers without Hydraulic Fr...,2020,0.00,0.0,0.0,SubpartW
3,NaN,1012154,580 - San Juan Basin Gathering/Boosting,Dehydrators [98.236(e)],2018,94270.75,1223.8,0.0,SubpartW
4,NaN,1004849,CONROE NATURAL GAS PROCESSING PLANT,Reciprocating Compressors [98.236(p)],2015,1825.50,4.4,0.0,SubpartW
...,...,...,...,...,...,...,...,...,...
503055,NaN,1005688,STATION #7 - MOUNTAINAIR CMP STN,Natural Gas Driven Pneumatic Pumps [98.236(c)],2018,0.00,0.0,0.0,SubpartW
503056,NaN,1007441,Eunice Gas Plant,Combustion Equipment at Onshore Petroleum and ...,2021,0.00,0.0,0.0,SubpartW
503057,395 - Williston Basin,1010002,Murex Petroleum Corporation - Williston Basin 395,Completions and Workovers without Hydraulic Fr...,2019,0.00,0.0,0.0,SubpartW
503058,NaN,1012624,Repsol Pipelines LLC - Eagle Ford,Onshore Production [98.236(aa)(1)],2016,0.00,0.0,0.0,SubpartW


In [1502]:
# #w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]
# temp_df = w_df[(w_df['reporting_year']==2022) & (w_df['facility_id']==1000435)]#[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']] = temp_df[['total_co2_emissions','total_ch4_emissions','total_n2o_emissions']].fillna(0)
# temp_df['total_ghg_emissions'] = temp_df['total_co2_emissions'] + temp_df['total_ch4_emissions'] + temp_df['total_n2o_emissions']
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
# temp_df = temp_df[temp_df['total_ghg_emissions']>0]
# temp_df[['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

## ConCat c_df and w_df

In [1503]:
df = pd.concat([c_df,w_df], ignore_index=True)

In [1504]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,771838,0.000000,9564,int64
1,facility_name,0,771838,0.000000,10447,object
2,state,503060,268778,65.176889,54,object
3,reporting_year,0,771838,0.000000,14,int64
4,Unit Name,503060,268778,65.176889,21995,object
5,reporting_category,9103,762735,1.179393,117,object
6,maximum_rated_heat_input_capacity,556592,215246,72.112542,6996,float64
7,total_co2_emissions,0,771838,0.000000,181252,float64
8,total_ch4_emissions,0,771838,0.000000,55506,float64
9,total_n2o_emissions,0,771838,0.000000,15814,float64


In [1505]:
# df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
# df = df[df['total_ghg_emissions']>0]

In [1506]:
len(df)

771838

## Rename Reporting Category

In [1507]:
reporting_category_names_df = pd.read_csv('reporting_categories.csv')
df = pd.merge(df, reporting_category_names_df, how='left',on=['reporting_category'])
df = df.drop('reporting_category',axis=1)
df = df.rename(columns={'REVISED_COLUMNNAME':'reporting_category'})

## validate before substitutions

In [1508]:
facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()

validation_before_sub_df = df
validation_before_sub_df = validation_before_sub_df[validation_before_sub_df.facility_id.isin(facilities_in_required_subsectors)]

validation_before_sub_df['total_ghg_emissions'] = validation_before_sub_df['total_co2_emissions'] + validation_before_sub_df['total_ch4_emissions'] + validation_before_sub_df['total_n2o_emissions']
validation_before_sub_df = validation_before_sub_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
validation_before_sub_df=validation_before_sub_df.groupby(['facility_id','reporting_year']).sum().reset_index()
validation_before_sub_df = pd.merge(validation_before_sub_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])

validation_before_sub_df = validation_before_sub_df[validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']>0]
validation_before_sub_df['diff'] = (validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']-validation_before_sub_df['total_ghg_emissions']).abs()
validation_before_sub_df['diff_percent_GHG']=100*validation_before_sub_df['diff']/validation_before_sub_df['FACILITY_WIDE_GHG_QUANTITY']
validation_before_sub_df.to_csv('validation_before_sub_df.csv', index=False)
validation_before_sub_df

C:\Users\shankze\AppData\Local\Temp\ipykernel_8412\1099216245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_before_sub_df['total_ghg_emissions'] = validation_before_sub_df['total_co2_emissions'] + validation_before_sub_df['total_ch4_emissions'] + validation_before_sub_df['total_n2o_emissions']


,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY,diff,diff_percent_GHG
0,1000355,2011,7816.60,9896.5476,14.8039,17727.9515,17728,0.0485,0.000274
1,1000355,2012,68969.16,82493.2142,56.6200,151518.9942,151519,0.0058,0.000004
2,1000355,2013,59015.60,103953.4000,44.7000,163013.7000,163014,0.3,0.000184
3,1000355,2014,116249.20,126776.7000,90.7000,243116.6000,243117,0.4,0.000165
4,1000355,2015,172100.20,182616.2500,134.1000,354850.5500,354851,0.45,0.000127
...,...,...,...,...,...,...,...,...,...
8164,1014903,2023,29273.30,1565.2500,51.5540,30890.1040,30890,0.104,0.000337
8165,1014906,2023,47690.50,7281.0000,40.2300,55011.7300,55011,0.73,0.001327
8166,1014909,2023,408369.20,6451.7500,574.2460,415395.1960,415395,0.196,0.000047
8167,1014920,2023,1742.40,23845.5000,0.5960,25588.4960,25589,0.504,0.00197


In [1509]:
validation_df[validation_df.diff_percent_GHG > 10]

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY,diff,diff_percent_GHG
0,1000355,2011,3477.100000,9793.9286,12.5929,13283.621500,17728,4444.3785,25.069825
1,1000355,2012,32826.460000,35431.3094,38.4516,68296.221000,151519,83222.779,54.925639
2,1000355,2013,57883.300000,28165.4000,43.5000,86092.200000,163014,76921.8,47.187235
3,1000355,2014,109641.500000,32792.9000,88.0000,142522.400000,243117,100594.6,41.377032
13,1000435,2011,120035.700000,58370.4761,89.1117,178495.287800,221022,42526.7122,19.240941
...,...,...,...,...,...,...,...,...,...
6545,1011773,2014,15151.300000,640.4000,35.3000,15827.000000,25157,9330.0,37.087093
6646,1011894,2014,21161.403076,12356.0000,18.6000,33536.003076,44523,10986.996924,24.677126
6707,1011962,2012,35399.500000,71382.7381,24.7051,106806.943200,121587,14780.0568,12.155952
6708,1011962,2013,37296.000000,71666.2000,24.6000,108986.800000,138574,29587.2,21.351191


## Concat Pneumatic Devices from detailed dataset

In [1510]:
len(df[df.reporting_category == 'NG_PneumaticDevices'])

24595

In [1511]:
df.columns

Index(['facility_id', 'facility_name', 'state', 'reporting_year', 'Unit Name',
       'maximum_rated_heat_input_capacity', 'total_co2_emissions',
       'total_ch4_emissions', 'total_n2o_emissions', 'subpart',
       'basin_associated_with_facility', 'reporting_category'],
      dtype='object')

In [1512]:
# Drop Pneumatic Devices from original dataset
df = df[df.reporting_category != 'NG_PneumaticDevices']

# add them from this csv file
pneumatic_devices_df = pd.read_csv('ef_w_ngpneumatic_dev_units.csv')
pneumatic_devices_df = pneumatic_devices_df[['facility_id','reporting_year','facility_name','estimated_number_of_hours','pneumatic_device_type','total_count','pneum_device_type_tot_co2_emm','total_ch4_emissions']]
pneumatic_devices_df=pneumatic_devices_df.rename(columns={'pneumatic_device_type':'Unit Name','estimated_number_of_hours':'pneu_est_no_of_hours','total_count':'pneu_dev_total_count','pneum_device_type_tot_co2_emm':'total_co2_emissions'})
pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']] = pneumatic_devices_df[['total_co2_emissions','total_ch4_emissions']].fillna(0)
pneumatic_devices_df['total_ch4_emissions'] = pneumatic_devices_df['total_ch4_emissions'] * 25
pneumatic_devices_df['total_n2o_emissions'] = 0
pneumatic_devices_df['reporting_category'] = 'NG_PneumaticDevices'
pneumatic_devices_df['subpart'] = 'SubpartW'

df = pd.concat([df,pneumatic_devices_df], ignore_index=True)
df = df.drop(columns=['state'])

## Concat AssociatedGasVentingFlaring from detailed dataset

In [1513]:
len(df[df.reporting_category == 'AssociatedGasVentingFlaring'])

22328

In [1514]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AssociatedGasVentingFlaring']

In [1515]:
# add them from this csv file
venting_flaring_df = pd.read_csv('ef_w_associated_ng_units.csv')

In [1516]:
venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']] = venting_flaring_df[['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions']].fillna(0)
venting_flaring_df['total_co2_emissions'] =  venting_flaring_df['basin_ag_flaring_co2_emissions'] + venting_flaring_df['basin_ag_venting_co2_emissions']
venting_flaring_df['total_ch4_emissions'] =  venting_flaring_df['venting_ch4_emissions'] + venting_flaring_df['flaring_ch4_emissions']
venting_flaring_df = venting_flaring_df.drop(['basin_ag_flaring_co2_emissions','basin_ag_venting_co2_emissions','venting_ch4_emissions','flaring_ch4_emissions'],axis=1)
venting_flaring_df = venting_flaring_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
venting_flaring_df[['total_n2o_emissions']] = venting_flaring_df[['total_n2o_emissions']].fillna(0)
venting_flaring_df['total_ch4_emissions'] = venting_flaring_df['total_ch4_emissions'] * 25
venting_flaring_df['total_n2o_emissions'] = venting_flaring_df['total_n2o_emissions'] * 298
venting_flaring_df['reporting_category'] = 'AssociatedGasVentingFlaring'
venting_flaring_df['subpart'] = 'SubpartW'
venting_flaring_df = venting_flaring_df.drop(['facility_name','industry_segment','wells_two_year_delay'], axis=1)
df = pd.concat([df,venting_flaring_df], ignore_index=True)

## Concat AtmosphericTanks from detailed dataset

In [1517]:
len(df[df.reporting_category == 'AtmosphericTanks'])

22328

In [1518]:
# Drop AssociatedGasVentingFlaring from original dataset
df = df[df.reporting_category != 'AtmosphericTanks']

In [1519]:
# add them from this csv file
athmospheric_tanks_df = pd.read_csv('ef_w_atm_stg_tanks_calc1or2.csv')

In [1520]:
athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']] = athmospheric_tanks_df[['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions']].fillna(0)
athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']] = athmospheric_tanks_df[['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions']].fillna(0)

athmospheric_tanks_df['total_co2_emissions'] =  athmospheric_tanks_df['dump_valve_co2_emissions'] + athmospheric_tanks_df['flaring_co2_emissions'] + athmospheric_tanks_df['vapor_recovery_co2_emissions'] + athmospheric_tanks_df['venting_co2_emissions']
athmospheric_tanks_df['total_ch4_emissions'] =  athmospheric_tanks_df['dump_valve_ch4_emissions'] + athmospheric_tanks_df['flaring_ch4_emissions'] + athmospheric_tanks_df['vapor_recovery_ch4_emissions']+athmospheric_tanks_df['venting_ch4_emissions'] 
athmospheric_tanks_df = athmospheric_tanks_df.rename(columns={'flaring_n2o_emissions':'total_n2o_emissions'})
athmospheric_tanks_df[['total_n2o_emissions']] = athmospheric_tanks_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_co2_emissions','flaring_co2_emissions','vapor_recovery_co2_emissions','venting_co2_emissions'],axis=1)
athmospheric_tanks_df = athmospheric_tanks_df.drop(['dump_valve_ch4_emissions','flaring_ch4_emissions','vapor_recovery_ch4_emissions','venting_ch4_emissions'],axis=1)
athmospheric_tanks_df['total_ch4_emissions'] = athmospheric_tanks_df['total_ch4_emissions'] * 25
athmospheric_tanks_df['total_n2o_emissions'] = athmospheric_tanks_df['total_n2o_emissions'] * 298
athmospheric_tanks_df=athmospheric_tanks_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','annual_ch4_recovered','annual_co2_recovered','atmospheric_tank_count',
                                             'average_a_p_i_gravity','average_separator_pressure','average_separator_temperature',
                                             'hrs_dump_valves_did_not_close','not_on_well_pad_tank_count','num_seps_with_stuck_dump_val',
                                             'number_of_wells_send_oil','tanks_venting_to_atm','tanks_with_flaring','tanks_with_vapor_recovery',
'total_volume_of_oil']]
athmospheric_tanks_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_df['subpart'] = 'SubpartW'

In [1521]:
athmospheric_tanks_2_df = pd.read_csv('ef_w_atm_stg_tanks_calc3.csv')

athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']] = athmospheric_tanks_2_df[['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared']].fillna(0)

athmospheric_tanks_2_df['total_co2_emissions'] =  athmospheric_tanks_2_df['co2_emissions_from_flare'] + athmospheric_tanks_2_df['co2_emissions_not_flared']
athmospheric_tanks_2_df['total_ch4_emissions'] =  athmospheric_tanks_2_df['ch4_emissions_from_flare'] + athmospheric_tanks_2_df['ch4_emissions_not_flared']
athmospheric_tanks_2_df = athmospheric_tanks_2_df.drop(['co2_emissions_from_flare','co2_emissions_not_flared','ch4_emissions_from_flare','ch4_emissions_not_flared'],axis=1)
athmospheric_tanks_2_df = athmospheric_tanks_2_df.rename(columns={'n2o_emissions_from_flare':'total_n2o_emissions'})
athmospheric_tanks_2_df[['total_n2o_emissions']] = athmospheric_tanks_2_df[['total_n2o_emissions']].fillna(0)
athmospheric_tanks_2_df['total_ch4_emissions'] = athmospheric_tanks_2_df['total_ch4_emissions'] * 25
athmospheric_tanks_2_df['total_n2o_emissions'] = athmospheric_tanks_2_df['total_n2o_emissions'] * 298
athmospheric_tanks_2_df=athmospheric_tanks_2_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions',
                                                 'annual_oil_throughput','atmospheric_tank_count','fract_oil_throughput_flaring',
                                                'fract_throughput_with_vapor','tanks_with_flares','tanks_without_flares',
                                                'wells_with_seps','wells_without_seps']]
athmospheric_tanks_2_df['reporting_category'] = 'AtmosphericTanks'
athmospheric_tanks_2_df['subpart'] = 'SubpartW'

In [1522]:
df = pd.concat([df,athmospheric_tanks_df,athmospheric_tanks_2_df], ignore_index=True)
len(df)

814638

In [1523]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,814638,0.000000,9564,int64
1,facility_name,73111,741527,8.974661,10447,object
2,reporting_year,0,814638,0.000000,14,int64
3,Unit Name,506920,307718,62.226412,21998,object
4,maximum_rated_heat_input_capacity,599392,215246,73.577712,6996,float64
5,total_co2_emissions,0,814638,0.000000,207055,float64
6,total_ch4_emissions,0,814638,0.000000,79052,float64
7,total_n2o_emissions,0,814638,0.000000,24956,float64
8,subpart,0,814638,0.000000,2,object
9,basin_associated_with_facility,680127,134511,83.488249,44,object


## Create total_ghg_emissions column

In [1524]:
df['total_ghg_emissions'] = df['total_co2_emissions'] + df['total_ch4_emissions'] + df['total_n2o_emissions']
df = df[df['total_ghg_emissions']>0]

In [1525]:
len(df)

446390

In [1526]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,446390,0.000000,9550,int64
1,facility_name,62592,383798,14.021819,10431,object
2,reporting_year,0,446390,0.000000,14,int64
3,Unit Name,169264,277126,37.918412,21539,object
4,maximum_rated_heat_input_capacity,241040,205350,53.997625,6937,float64
5,total_co2_emissions,0,446390,0.000000,207055,float64
6,total_ch4_emissions,0,446390,0.000000,79052,float64
7,total_n2o_emissions,0,446390,0.000000,24956,float64
8,subpart,0,446390,0.000000,2,object
9,basin_associated_with_facility,396485,49905,88.820314,44,object


In [1527]:
df[(df['reporting_year']==2022) & (df['facility_id']==1000435)][['reporting_category','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]

,reporting_category,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions
446004,EquipmentLeaks,148.60000,40883.750,0.000,41032.35000
528241,CombustionEquipment,25656.00000,841.750,19.072,26516.82200
548113,ReciprocatingCompressors,0.70000,110.000,0.000,110.70000
563930,Dehydrators,3.50000,655.750,0.000,659.25000
617438,NG_PneumaticPumps,5.50000,1522.500,0.000,1528.00000
661048,WellVenting,3.00000,368.500,0.000,371.50000
712519,NG_PneumaticDevices,3.00000,811.750,0.000,814.75000
717056,NG_PneumaticDevices,1.90000,527.500,0.000,529.40000
729877,NG_PneumaticDevices,18.50000,5096.500,0.000,5115.00000
748196,AtmosphericTanks,0.02279,26.288,0.000,26.31079


## Filter df to only required subsectors

In [1528]:
facilities_in_required_subsectors = facilities_master_df['facility_id'].unique()
print(len(df))
df = df[df.facility_id.isin(facilities_in_required_subsectors)]
print(len(df))

446390
97613


## Join Facilities Metadata

In [1529]:
df = pd.merge(df,facilities_master_df,how='left',on=['facility_id','reporting_year'])


In [1530]:
#df

In [1531]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,97613,0.000000,1148,object
1,facility_name,48627,48986,49.816111,1282,object
2,reporting_year,0,97613,0.000000,13,object
3,Unit Name,84946,12667,87.023245,459,object
4,maximum_rated_heat_input_capacity,91593,6020,93.832789,766,float64
5,total_co2_emissions,0,97613,0.000000,41465,float64
6,total_ch4_emissions,0,97613,0.000000,46699,float64
7,total_n2o_emissions,0,97613,0.000000,10893,float64
8,subpart,0,97613,0.000000,2,object
9,basin_associated_with_facility,62601,35012,64.131827,44,object


In [1532]:
#df = df[df['GHG FACILITY_WIDE_GHG_QUANTITY']>0]

## Validate GHG Quantity for facility

In [1533]:

validation_df = df
validation_df = validation_df[['facility_id','reporting_year','total_co2_emissions','total_ch4_emissions','total_n2o_emissions','total_ghg_emissions']]
validation_df=validation_df.groupby(['facility_id','reporting_year']).sum().reset_index()
#validation_df

In [1534]:
validation_df = pd.merge(validation_df,facilities_master_df[['facility_id','reporting_year','FACILITY_WIDE_GHG_QUANTITY']],how='left',on=['facility_id','reporting_year'])
#validation_df

In [1535]:
validation_df['diff'] = (validation_df['FACILITY_WIDE_GHG_QUANTITY']-validation_df['total_ghg_emissions']).abs()
validation_df['diff_percent_GHG']=100*validation_df['diff']/validation_df['FACILITY_WIDE_GHG_QUANTITY']
validation_df

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY,diff,diff_percent_GHG
0,1000355,2011,3477.100000,9793.928600,12.592900,13283.621500,17728,4444.3785,25.069825
1,1000355,2012,32826.460000,35431.309400,38.451600,68296.221000,151519,83222.779,54.925639
2,1000355,2013,57883.300000,28165.400000,43.500000,86092.200000,163014,76921.8,47.187235
3,1000355,2014,109641.500000,32792.900000,88.000000,142522.400000,243117,100594.6,41.377032
4,1000355,2015,172100.229209,182616.353975,134.043320,354850.626504,354851,0.373496,0.000105
...,...,...,...,...,...,...,...,...,...
8081,1014903,2023,29273.300000,1565.250000,51.554000,30890.104000,30890,0.104,0.000337
8082,1014906,2023,47690.448409,7281.108800,39.987130,55011.544339,55011,0.544339,0.00099
8083,1014909,2023,408369.191967,6451.547665,574.273684,415395.013316,415395,0.013316,0.000003
8084,1014920,2023,1742.387125,23845.602950,0.668116,25588.658191,25589,0.341809,0.001336


In [1536]:
validation_df[validation_df.diff_percent_GHG > 10]

,facility_id,reporting_year,total_co2_emissions,total_ch4_emissions,total_n2o_emissions,total_ghg_emissions,FACILITY_WIDE_GHG_QUANTITY,diff,diff_percent_GHG
0,1000355,2011,3477.100000,9793.9286,12.5929,13283.621500,17728,4444.3785,25.069825
1,1000355,2012,32826.460000,35431.3094,38.4516,68296.221000,151519,83222.779,54.925639
2,1000355,2013,57883.300000,28165.4000,43.5000,86092.200000,163014,76921.8,47.187235
3,1000355,2014,109641.500000,32792.9000,88.0000,142522.400000,243117,100594.6,41.377032
13,1000435,2011,120035.700000,58370.4761,89.1117,178495.287800,221022,42526.7122,19.240941
...,...,...,...,...,...,...,...,...,...
6545,1011773,2014,15151.300000,640.4000,35.3000,15827.000000,25157,9330.0,37.087093
6646,1011894,2014,21161.403076,12356.0000,18.6000,33536.003076,44523,10986.996924,24.677126
6707,1011962,2012,35399.500000,71382.7381,24.7051,106806.943200,121587,14780.0568,12.155952
6708,1011962,2013,37296.000000,71666.2000,24.6000,108986.800000,138574,29587.2,21.351191


In [1537]:
validation_df.to_csv('output_validation.csv', index=False)

In [1538]:
generate_column_info(df)

,ColumnName,NullCount,NonNullCount,NullPercent,UniqueValueCount,DataType
0,facility_id,0,97613,0.000000,1148,object
1,facility_name,48627,48986,49.816111,1282,object
2,reporting_year,0,97613,0.000000,13,object
3,Unit Name,84946,12667,87.023245,459,object
4,maximum_rated_heat_input_capacity,91593,6020,93.832789,766,float64
5,total_co2_emissions,0,97613,0.000000,41465,float64
6,total_ch4_emissions,0,97613,0.000000,46699,float64
7,total_n2o_emissions,0,97613,0.000000,10893,float64
8,subpart,0,97613,0.000000,2,object
9,basin_associated_with_facility,62601,35012,64.131827,44,object


In [1539]:
df.to_csv('output.csv', index=False)

In [1540]:
#AssociatedGasVentingFlaring

# AtmosphericTanks